In [4]:
%run Functions.py

In [5]:
def computeErrorRate(PhraseInList, PhraseOutList, Language):
    
    # levenshtein distance at word level
    dist_word = list()
    
    for ind, phraseIn_wTime in enumerate(PhraseInList):
        phraseIn = phraseIn_wTime[1]
        phraseOut = PhraseOutList[ind][1]
        
        # initialize the stemmer
        if Language == 'Danish':
            words_stemmer = snowball.DanishStemmer()
        else:
            words_stemmer = snowball.EnglishStemmer()
            
            
        roots_phraseIn = list()
        roots_phraseOut = list()
                    
        for word in phraseIn.split():
            # find roots of words in stimulation phrase - 
            roots_phraseIn.append(words_stemmer.stem(word))

        for word in phraseOut.split():
            roots_phraseOut.append(words_stemmer.stem(word))
                        
        #print(roots_phraseStim)
        #print('and')
        #print(roots_phraseUser)
                    
        #print(distance.levenshtein(roots_phraseStim, roots_phraseUser))
        dist_word.append(distance.levenshtein(roots_phraseIn, roots_phraseOut)/max(len(roots_phraseIn), \
                                                                                              len(roots_phraseOut)))

    # levenshtein distance at character level
    dist_char = list()
    
    for n in range(0,len(PhraseInList)):
        dist_char.append((levenshteinDist(PhraseInList[n][1], PhraseOutList[n][1]))/max(len(PhraseInList[n][1]), \
                                                                                        len(PhraseOutList[n][1])))
        
    ErrorRate_trial = list(0.5*np.array(dist_char) + 0.5*np.array(dist_word))
    
    
    return ErrorRate_trial   

In [6]:
metricComputed = 'ErrorRate'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\Subject_Block_Session_Trial_' + metricComputed +  '.xlsx'


for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        if 'ls2\\4\\' not in root:
            continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None or scratchPad is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # find the language of the data:
            if 'Hvad er det fulde navn på dit universitet?' in phraseLog_new[1][1]: 
                language = 'Danish'
            else:
                language = 'English'            
            
            phraseStim_reduced = stimPhrasesEdit(phraseLog_new, root)
            
            # if it is the 2nd part of the session, picture is not described:
            if '2ndPart' in root:
                picture = 'not_described'
            else:
                picture = 'described'
            
            phraseUserEnd_reduced = scratchPadPhraseEdit(scratchPad_new, subjName, root, picture)
            
            errorRate = computeErrorRate(phraseStim_reduced, phraseUserEnd_reduced, language)
            
            
            if '1stPart' in root:
                errorRate1 = errorRate
                continue
            
            
            
            if '2ndPart' in root:
                errorRate2 = errorRate
                errorRate = errorRate1 + errorRate2
                
                
                errorRate1 = list()
                
                
            dataToSave = DataForEveryTrial()
            dataToSave.subjectID = subjAndSessionName.split('__')[0]
            dataToSave.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave.sessionNumber = subjAndSessionName[-1]
            dataToSave.variable = metricComputed
            dataToSave.dataForTrial = errorRate
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            #dataToSave.AddToFile()
            
            

subject path E:\Data\Data\ls2\4\2019-02-18-10-09-22_1
subject and session name:  ls2__4__2019-02-18-10-09-22_1
  subjectID block session  trial  ErrorRate
0       ls2     4       1      0   0.000000
1       ls2     4       1      1   0.000000
2       ls2     4       1      2   0.000000
3       ls2     4       1      3   0.000000
4       ls2     4       1      4   0.145455
subject path E:\Data\Data\ls2\4\2019-02-18-10-28-35_2
subject and session name:  ls2__4__2019-02-18-10-28-35_2
  subjectID block session  trial  ErrorRate
0       ls2     4       2      0   0.888889
1       ls2     4       2      1   0.883117
2       ls2     4       2      2   0.854012
3       ls2     4       2      3   0.834968
